<a href="https://www.kaggle.com/code/rajatraj0502/cafa-5-protein-function-prediction?scriptVersionId=141738711" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo


In [2]:
# Load the first few lines of the IA.txt file
ia_file_path = os.path.join('/kaggle/input/cafa-5-protein-function-prediction/IA.txt')

with open(ia_file_path, 'r') as file:
    ia_contents = [next(file) for _ in range(5)]

ia_contents


['GO:0000001\t0.0\n',
 'GO:0000002\t3.10383581098454\n',
 'GO:0000003\t3.4394041159928754\n',
 'GO:0000011\t0.056583528366367424\n',
 'GO:0000012\t6.400377192572712\n']

In [3]:
# List the contents of the Train folder
train_dir = os.path.join('/kaggle/input/cafa-5-protein-function-prediction/Train')
train_files = os.listdir(train_dir)
train_files


['train_terms.tsv',
 'train_sequences.fasta',
 'train_taxonomy.tsv',
 'go-basic.obo']

In [4]:
from Bio import SeqIO

# Load the first few entries from the train_sequences.fasta file
train_sequences_file_path = os.path.join(train_dir, 'train_sequences.fasta')
train_sequences = list(SeqIO.parse(train_sequences_file_path, "fasta"))

# Display the first 3 entries
train_sequences[:3]


[SeqRecord(seq=Seq('MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLK...FIY'), id='P20536', name='P20536', description='P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1', dbxrefs=[]),
 SeqRecord(seq=Seq('MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKLLDGLVPDQ...VSK'), id='O73864', name='O73864', description='O73864 sp|O73864|WNT11_DANRE Protein Wnt-11 OS=Danio rerio OX=7955 GN=wnt11 PE=2 SV=1', dbxrefs=[]),
 SeqRecord(seq=Seq('MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLPGPGQTSGA...DAF'), id='O95231', name='O95231', description='O95231 sp|O95231|VENTX_HUMAN Homeobox protein VENTX OS=Homo sapiens OX=9606 GN=VENTX PE=1 SV=1', dbxrefs=[])]

In [5]:
import pandas as pd

# Load the first few lines of the train_taxonomy.tsv file
train_taxonomy_file_path = os.path.join(train_dir, 'train_taxonomy.tsv')
train_taxonomy_sample = pd.read_csv(train_taxonomy_file_path, sep='\t', nrows=5)

train_taxonomy_sample


,EntryID,taxonomyID
0,Q8IXT2,9606
1,Q04418,559292
2,A8DYA3,7227
3,Q9UUI3,284812
4,Q57ZS4,185431


In [6]:
# Load the first few lines of the train_terms.tsv file
train_terms_file_path = os.path.join(train_dir, 'train_terms.tsv')
train_terms_sample = pd.read_csv(train_terms_file_path, sep='\t', nrows=5)

train_terms_sample


,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


In [7]:
# List the contents of the Test (Targets) folder
test_targets_dir = os.path.join('/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)')
test_targets_files = os.listdir(test_targets_dir)
test_targets_files


['testsuperset.fasta', 'testsuperset-taxon-list.tsv']

In [8]:
from Bio import SeqIO

# Load the amino acid sequences from the train_sequences.fasta file
train_sequences_file_path = '/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta'
train_sequences = list(SeqIO.parse(train_sequences_file_path, "fasta"))

# Display the first entry
print(train_sequences[0])


ID: P20536
Name: P20536
Description: P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1
Number of features: 0
Seq('MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLK...FIY')


In [9]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Load the GO terms from the train_terms.tsv file
train_terms_file_path = '/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv'
train_terms = pd.read_csv(train_terms_file_path, sep='\t')

# Encode the GO terms using multi-label binarization
mlb = MultiLabelBinarizer()
train_terms_encoded = mlb.fit_transform(train_terms['term'])

# Display the first few rows of the encoded GO terms
print(train_terms_encoded[:5])


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[[1 1 1 0 0 1 0 0 1 0 1 1 1]
 [1 0 0 1 1 1 1 0 0 0 1 1 1]
 [1 0 1 1 0 1 0 1 0 0 1 1 1]
 [1 0 1 0 1 0 0 1 0 0 1 1 1]
 [1 0 0 1 0 1 1 1 0 0 1 1 1]]


In [10]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

# Load the GO terms from the train_terms.tsv file
train_terms_file_path = '/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv'
train_terms = pd.read_csv(train_terms_file_path, sep='\t')

# Encode the GO terms using one-hot encoding
lb = LabelBinarizer()
train_terms_encoded = lb.fit_transform(train_terms['term'])

# Display the first few rows of the encoded GO terms
print(train_terms_encoded[:5])


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
# Create a list of all unique amino acids in the training data
vocab = list(set(''.join(str(record.seq) for record in train_sequences)))

# Get the maximum length of the amino acid sequences in the training data
max_length = max(len(record.seq) for record in train_sequences)


In [12]:
# Create a list of all unique GO terms in the training data
go_terms = train_terms['term'].unique()


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Define the model
model = Sequential([
    Embedding(input_dim=len(vocab), output_dim=128, input_length=max_length),
    Conv1D(128, 3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(256, activation='relu'),
    Dense(len(go_terms), activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy')

# Display the model's architecture
model.summary()


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 35375, 128)        3200      
                                                                 
 conv1d (Conv1D)             (None, 35373, 128)        49280     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 31466)             8086762   
                                                                 
Total params: 8,172,266
Trainable params: 8,172,266
Non-trainable params: 0
______________________________________________

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_sequences(sequences):
    # Tokenize the sequences
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(sequences)
    sequences_tokenized = tokenizer.texts_to_sequences(sequences)
    
    # Pad the sequences
    max_length = max(len(seq) for seq in sequences_tokenized)
    sequences_padded = pad_sequences(sequences_tokenized, maxlen=max_length, padding='post')
    
    # One-hot encode the sequences
    X = np.array([np.eye(len(tokenizer.word_index))[seq - 1] for seq in sequences_padded])
    
    return X

# Example usage:
# X_test = preprocess_sequences(test_sequences)


In [15]:
from Bio import SeqIO

test_sequences = list(SeqIO.parse("/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta", "fasta"))


def preprocess_sequences(sequences):
    # Your code here to preprocess the sequences
    pass

X_test = preprocess_sequences(test_sequences)



In [16]:
go_terms = []  # list of all GO terms
# ... populate go_terms list ...

go_term_index = {go_term: i for i, go_term in enumerate(go_terms)}


In [17]:
# Create the submission file
with open('submission.tsv', 'w') as f:
    for i, sequence in enumerate(test_sequences):
        for go_term, index in go_term_index.items():
            if y_test_pred_bin[i, index] == 1:
                f.write(f'{sequence.id}\t{go_term}\t{y_test_pred[i, index]:.3f}\n')